In [1]:
import torch
import torch.nn as nn
import numpy as np
from option.coordinet_option import CoordiNetOption
from utils.utils import *
from model.coordinet import CoordiNetSystem
from dataset.cambridge import CambridgeDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import os, sys
import time
import pickle
import matplotlib.pyplot as plt

In [2]:
opt_path='runs/coordinet/church_train_synthesis2/opt.pkl'
with open(opt_path, 'rb') as f:
    opt = pickle.load(f)
# dataset
train_dataset = CambridgeDataset(opt.data_root_dir, opt.scene, 'train',320,300)
valid_dataset = CambridgeDataset(opt.data_root_dir, opt.scene, 'valid',320,300)
valid_dataloader = DataLoader(valid_dataset, batch_size=opt.batch_size, num_workers=16, shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=opt.batch_size, num_workers=16, shuffle=False)

load image and pose data of scene "StMarysChurch", split: train
load train data:   0%|          | 0/1487 [00:00<?, ?it/s]

load train data: 100%|██████████| 1487/1487 [00:00<00:00, 12220.67it/s]
load image and pose data of scene "StMarysChurch", split: valid
load test data: 100%|██████████| 530/530 [00:00<00:00, 12279.22it/s]


In [3]:
# init model
ckpt_path = 'runs/coordinet/church_train_synthesis2/ckpt_epoch129_steps1006_angle5.0642_t1.1015.pkl'
opt.var_min=[0.25,0.25,0.25,0.0314]
coordinet = CoordiNetSystem(opt.feature_dim,opt.W,backbone_path=opt.backbone_path)
ckpt = torch.load(ckpt_path)
coordinet.load_state_dict(ckpt['coordinet'])
print('load model from {}'.format(ckpt_path))
coordinet = coordinet.cuda()
coordinet.eval();

Loaded pretrained weights for efficientnet-b3
load model from runs/coordinet/church_train_synthesis2/ckpt_epoch129_steps1006_angle5.0642_t1.1015.pkl


In [4]:
# valid: center crop from test set
t_diffs = []
angle_diffs = []
ts=[]
for imgs, poses in tqdm(valid_dataloader, total=len(valid_dataloader), file=sys.stdout):
    imgs = imgs.cuda()
    poses = poses.cuda()
    with torch.no_grad():
        t, q, _, losses = coordinet.forward(imgs, cal_loss=True, y_pose=poses)
    ts.append(t.cpu().numpy())
    t_diffs.append(losses['t_diff'].cpu().numpy())
    angle_diffs.append(losses['angle_diff'].cpu().numpy())
t_diffs = np.hstack(t_diffs)
angle_diffs = np.hstack(angle_diffs) / np.pi * 180
angle_diff = np.mean(angle_diffs)
t_diff = np.mean(np.hstack(t_diffs))
print('[valid] angle_diff:%.4f°, t_diff:%.4fm' % (angle_diff, t_diff))

ts=np.vstack(ts)
# save
with open('./valid_diff.txt', 'w') as f:
    for i in range(len(t_diffs)):
        GT_t=valid_dataset.test_poses[i][:3,3]
        t=ts[i]
        f.write('%.4fm %.4f° [GT:(%.3f,%.3f,%.3f)] [Re:(%.3f,%.3f,%.3f)]\n' % (t_diffs[i], angle_diffs[i],*list(GT_t),*list(t)))

100%|██████████| 53/53 [00:08<00:00,  6.31it/s]
[valid] angle_diff:5.0642°, t_diff:1.1015m


In [4]:
# test: uniform num crops from test set
t_diffs = []
angle_diffs = []
ts=[]
num_crops=128
for i in tqdm(range(len(valid_dataset.test_paths))):
    imgs,poses=valid_dataset.uniform_crop(valid_dataset.test_paths[i],num_crops,valid_dataset.test_poses[i])
    it,it_diffs,iangle_diffs=[],[],[]
    for j in range(0,len(imgs),opt.batch_size):
        img,pose=imgs[j:j+opt.batch_size],poses[j:j+opt.batch_size]
        img,pose=img.cuda(),pose.cuda()
        with torch.no_grad():
            t, q, _, losses = coordinet.forward(img, cal_loss=True, y_pose=pose)
        it.append(t.cpu().numpy())
        it_diffs.append(losses['t_diff'].cpu().numpy())
        iangle_diffs.append(losses['angle_diff'].cpu().numpy())
    it = np.vstack(it)
    it_diffs = np.hstack(it_diffs)
    iangle_diffs = np.hstack(iangle_diffs) / np.pi * 180
    ts.append(np.mean(it,axis=0))
    t_diffs.append(np.mean(it_diffs))
    angle_diffs.append(np.mean(iangle_diffs))
t_diffs = np.hstack(t_diffs)
angle_diffs = np.hstack(angle_diffs)
t_diff=np.mean(t_diffs)
angle_diff=np.mean(angle_diffs)
print('[test] angle_diff:%.4f°, t_diff:%.4fm' % (angle_diff, t_diff))
ts=np.vstack(ts)
# save
with open('./test_diff.txt', 'w') as f:
    for i in range(len(t_diffs)):
        GT_t=valid_dataset.test_poses[i][:3,3]
        t=ts[i]
        f.write('%.4fm %.4f° [GT:(%.3f,%.3f,%.3f)] [Re:(%.3f,%.3f,%.3f)]\n' % (t_diffs[i], angle_diffs[i],*list(GT_t),*list(t)))

100%|██████████| 530/530 [04:56<00:00,  1.79it/s]

[test] angle_diff:5.8118°, t_diff:1.1466m


In [5]:
# train
coordinet.eval()
t_diffs = []
angle_diffs = []
for imgs, poses in tqdm(train_dataloader, total=len(train_dataloader), file=sys.stdout):
    imgs = imgs.cuda()
    poses = poses.cuda()
    with torch.no_grad():
        t, q, _, losses = coordinet.forward(imgs, cal_loss=True, y_pose=poses)
    t_diffs.append(losses['t_diff'].cpu().numpy())
    angle_diffs.append(losses['angle_diff'].cpu().numpy())
t_diffs = np.hstack(t_diffs)
angle_diffs = np.hstack(angle_diffs) / np.pi * 180
angle_diff = np.mean(angle_diffs)
t_diff = np.mean(t_diffs)
print('[train] angle_diff:%.4f°, t_diff:%.4fm' % (angle_diff, t_diff))
# save
with open('./train_diff.txt', 'w') as f:
    for i in range(len(t_diffs)):
        f.write('%.4fm %.4f°\n' % (t_diffs[i], angle_diffs[i]))

100%|██████████| 149/149 [00:17<00:00,  8.34it/s]
[train] angle_diff:3.2010°, t_diff:0.6388m
